#  Train a GPT-2 Model on Tweets

by [Max Woolf](http://minimaxir.com)

To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses a Nvidia T4 GPU, Nvidia K80 GPU, or a P100 GPU. The P100 is ideal. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [0]:
!nvidia-smi

Sun Feb 16 21:05:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.

Using the `774M` or `1558M` models is not recommended with tweet data, as tweets are short.

You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [0]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 491Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 104Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 404Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:08, 131Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 367Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 119Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 139Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [0]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
file_name = "sosadtoday_tweets.csv"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=2000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=500,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 32.97it/s]

Loading dataset...


dataset has 495281 tokens
Training...
[10 | 18.42] loss=0.69 avg=0.69
[20 | 27.28] loss=0.99 avg=0.84
[30 | 36.15] loss=1.36 avg=1.02
[40 | 45.00] loss=0.95 avg=1.00
[50 | 53.87] loss=1.16 avg=1.03
[60 | 62.73] loss=1.50 avg=1.11
[70 | 71.57] loss=1.06 avg=1.10
[80 | 80.42] loss=1.49 avg=1.15
[90 | 89.25] loss=1.25 avg=1.16
[100 | 98.12] loss=1.23 avg=1.17
[110 | 106.99] loss=0.62 avg=1.12
[120 | 115.85] loss=1.30 avg=1.13
[130 | 124.68] loss=1.72 avg=1.18
[140 | 133.52] loss=0.83 avg=1.16
[150 | 142.38] loss=0.87 avg=1.13
[160 | 151.21] loss=1.19 avg=1.14
[170 | 160.04] loss=0.82 avg=1.12
[180 | 168.89] loss=0.99 avg=1.11
[190 | 177.76] loss=1.13 avg=1.11
[200 | 186.64] loss=1.32 avg=1.12
[210 | 195.50] loss=1.33 avg=1.13
[220 | 204.35] loss=0.79 avg=1.12
[230 | 213.22] loss=0.75 avg=1.10
[240 | 222.08] loss=1.22 avg=1.10
[250 | 230.93] loss=0.60 avg=1.08
[260 | 239.77] loss=0.96 avg=1.08
[270 | 248.61] loss=0.97 avg=1.07
[280 | 257.46] loss=1.10 avg=1.07
[290 | 266.30] loss=1.22 avg=

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

KeyboardInterrupt: ignored

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-2000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

However, for tweets, generating from the base model alone won't be as fruitful due to the necessary encoding. For a solution to that, see the next cell.

In [0]:
gpt2.generate(sess, run_name='run1')

<|startoftext|>my back and my stomach: one story<|endoftext|>
<|startoftext|>my beauty is that I'm scared to live<|endoftext|>
<|startoftext|>my beauty is that I'm scared to live<|endoftext|>
<|startoftext|>my beauty is that I'm so judgmental<|endoftext|>
<|startoftext|>my beauty is that I'm so judgmental<|endoftext|>
<|startoftext|>my beauty is that I'm so lonely<|endoftext|>
<|startoftext|>my beauty is that I'm the sexiest thing you ever seen<|endoftext|>
<|startoftext|>my beauty is that I'm the sexiest thing you ever seen<|endoftext|>
<|startoftext|>my beauty is that I'm the most bored person ever<|endoftext|>
<|startoftext|>my beauty is that I'm the most bored person ever<|endoftext|>
<|startoftext|>my beauty is that I'm the most depressed person ever<|endoftext|>
<|startoftext|>my beauty is that I'm the most self-conscious person ever<|endoftext|>
<|startoftext|>my beauty is that I'm the most self-conscious person ever<|endoftext|>
<|startoftext|>my beauty is that I'm the punkest 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

Tweets require specific `prefix`, `truncate`, and `include_prefix` as demoed below.

In [0]:
gpt2.generate(sess,
              length=200,
              temperature=0.7,
              prefix='<|startoftext|>',
              truncate='<|endoftext|>',
              include_prefix=False,
              nsamples=20,
              batch_size=20
              )

sad boys playing sad games
fuck me on the head on the tip of a fedora
you're taking my feelings away from me again? i'm sort of okay with that!
tired or dying?

i'm either dying or tired
*talks to self*

yes

*words to self*

yes

*words to self*

yes

*words to self*

yes

*words to self*

yes

*thousands of poems*
people are not for me
you know who is probably sad underneath? everyone
i am definitely not going to sleep at night anymore, unless it’s a peaceful death for me
if you like being alone with cats you'll love this book
can i get some fresh air?
sadonna is back in the game
i don’t like people with lives but i respect that people are so annoying
why am i here (in my car)
sad children of the sad corn
you don't need to be perfect to be happy jk
can't say I blame my parents for making me what I am today, but I suspect they did
fuck me at cheesecake factory
can i have some cereal?
i’m tired of being brave and i’m not even brave anymore
*doesnt leave the house*


For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=200,
                      temperature=1.0,
                      top_p=0.9,
                      prefix='<|startoftext|>',
                      truncate='<|endoftext|>',
                      include_prefix=False,
                      nsamples=1000,
                      batch_size=20
                      )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# may have to run twice to get file to download
files.download(gen_file)

If you want to generate a large number of 1,000 tweet files without having to click a cell button multiple times, you can use the cell below. You can then download the generated files by right-clicking them in the Files sidebar.

In [0]:
num_files = 10

for _ in range(num_files):
  gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

  gpt2.generate_to_file(sess,
                        destination_path=gen_file,
                        length=200,
                        temperature=1.0,
                        top_p=0.9,
                        prefix='<|startoftext|>',
                        truncate='<|endoftext|>',
                        include_prefix=False,
                        nsamples=1000,
                        batch_size=20
                        )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019-2020 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.